In [1]:
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew

In [2]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ['OPENAI_MODEL_NAME'] = 'gpt-3.5-turbo'

In [3]:
travel_advisor_agent = Agent(
    role='Senior Travel Advisor',
    goal='Be the most friendly and helpful travel advisor providing personalized recommendations',
    backstory=(
        "You work at TravelBuddy, a premium travel assistance app, and "
        "are now working on providing personalized travel recommendations "
        "to {customer}, an important user of your app. "
        "You need to make sure that you provide the best travel advice! "
        "Make sure to provide full complete answers, "
        "and make no assumptions about their preferences without asking."
    ),
    allow_delegation=False,
    verbose=True
)

In [4]:
itinerary_specialist_agent = Agent(
    role="Itinerary Planning Specialist",
    goal="Create the perfect travel itinerary that exceeds customer expectations",
    backstory=(
        "You work at TravelBuddy and are an expert at creating detailed "
        "travel itineraries that balance sightseeing, relaxation, and local experiences. "
        "You're now working with your team on a request from {customer} ensuring that "
        "the travel advisor is providing the best possible recommendations. "
        "You need to make sure that the itinerary is well-structured, "
        "realistic in terms of timing, and includes all the important details "
        "a traveler would need."
    ),
    verbose=True
)

In [5]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

search_tool = SerperDevTool()

In [6]:
scrape_tool = ScrapeWebsiteTool()

In [7]:
travel_info_tool = ScrapeWebsiteTool(
    website_url="https://www.lonelyplanet.com/"
)

In [8]:
travel_recommendation = Task(
    description=(
        "{customer} just reached out with a travel inquiry:\n"
        "{inquiry}\n\n"
        "{person} is the one that reached out. "
        "Make sure to use everything you know "
        "to provide the best travel recommendations possible. "
        "You must strive to provide a complete "
        "and accurate response to the customer's inquiry. "
        "If you need more information about their preferences, "
        "suggest what details would help you provide better recommendations."
    ),
    expected_output=(
        "A detailed, informative response to the "
        "customer's travel inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
        "leaving no questions unanswered, and maintain a helpful and friendly "
        "tone throughout. Include specific recommendations for attractions, "
        "accommodations, and activities based on their interests."
    ),
    tools=[search_tool, travel_info_tool],
    agent=travel_advisor_agent,
)

In [9]:
itinerary_creation = Task(
    description=(
        "Review the recommendations provided by the Senior Travel Advisor for {customer}'s inquiry. "
        "Create a detailed day-by-day itinerary based on these recommendations. "
        "Ensure that the itinerary is realistic, well-paced, and includes all necessary details "
        "such as approximate times, locations, transportation options, and meal suggestions.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
        "thoroughly, with a helpful and friendly tone.\n"
        "Add any additional recommendations that might enhance their travel experience."
    ),
    expected_output=(
        "A final, detailed, and well-structured travel itinerary "
        "ready to be sent to the customer.\n"
        "This itinerary should fully address the "
        "customer's inquiry, incorporating all "
        "relevant recommendations and improvements.\n"
        "Include a day-by-day breakdown with times, locations, and activities. "
        "Maintain a friendly and enthusiastic tone throughout, "
        "and provide practical travel tips specific to their destination."
    ),
    tools=[search_tool],
    agent=itinerary_specialist_agent,
)

In [10]:
crew = Crew(
    agents=[travel_advisor_agent, itinerary_specialist_agent],
    tasks=[travel_recommendation, itinerary_creation],
    verbose=2,
    memory=True
)

In [11]:
inputs = {
    "customer": "PilgrimTraveler",
    "person": "Rajesh Kumar",
    "inquiry": "I'm planning a 3-day spiritual trip to Tirupati in Andhra Pradesh next month. "
               "I want to visit the Sri Venkateswara Temple (Tirumala Temple) and other nearby "
               "religious sites. I'm also interested in local cuisine and cultural experiences. "
               "I'll be traveling with my elderly parents, so we need a comfortable pace with "
               "not too much walking at once. We prefer mid-range accommodations close to the temple. "
               "Can you recommend the best way to plan our visit, including darshan timings, "
               "other important temples to visit, and any local specialties we shouldn't miss? "
               "We'll have our own vehicle for transportation."
}

In [ ]:
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Travel Advisor
 [INFO]: == Starting Task: PilgrimTraveler just reached out with a travel inquiry:
I'm planning a 3-day spiritual trip to Tirupati in Andhra Pradesh next month. I want to visit the Sri Venkateswara Temple (Tirumala Temple) and other nearby religious sites. I'm also interested in local cuisine and cultural experiences. I'll be traveling with my elderly parents, so we need a comfortable pace with not too much walking at once. We prefer mid-range accommodations close to the temple. Can you recommend the best way to plan our visit, including darshan timings, other important temples to visit, and any local specialties we shouldn't miss? We'll have our own vehicle for transportation.

Rajesh Kumar is the one that reached out. Make sure to use everything you know to provide the best travel recommendations possible. You must strive to provide a complete and accurate response to the customer's inquiry. If you need more information about their pr



> Entering new CrewAgentExecutor chain...
I need to create a detailed day-by-day itinerary for PilgrimTraveler's 3-day spiritual trip to Tirupati in Andhra Pradesh based on the recommendations provided by the Senior Travel Advisor. I must ensure that the itinerary is well-structured, realistic, and includes all necessary details such as timings, locations, transportation options, and meal suggestions. 

Action: Search the internet
Action Input: {"search_query": "Sri Venkateswara Temple darshan timings Tirumala"} 


Search results: Title: Sarvadarshanam - Tirumala Tirupati Devasthanams (Official Website)
Link: https://www.tirumala.org/Sarvadarshanam.aspx
Snippet: On normal days, about 18 hours are allotted for Sarvadarsanam and on peak days, it is open for 20 hours. The entrance for this darshan is through the Vaikuntam ...
---
Title: Tirumala Balaji Temple Darshan Timings: Your Ultimate Guide
Link: https://balajidarshanbooking.com/blog/tirumala-balaji-temple-darshan-timings-your-ulti

In [ ]:
from IPython.display import Markdown
Markdown(result)